# Image to Parquet

Convert images to binary and save them into a parquet file.

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Dev").getOrCreate()
print(spark)